In [1]:
import os
import openai
import json
from dotenv import load_dotenv

def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data


load_dotenv()
openai.organization = os.getenv("OPENAI_ORG")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [19]:
def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def save_json(filename,data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)
    return

In [12]:
from IPython.display import JSON

# Load chunks
`chunks.json` is a list of strings with max tokens and overlap

In [14]:
json_data = load_json("website_data/export/chunks.json")
#JSON(json_data)
len(json_data)

419

## chunks truncation
for demo purpose, only first 10 chunks used

In [15]:
text_input = json_data[:10]

# Embeddings API call
trandforms Chunks into Embeddings

In [16]:
response = openai.Embedding.create(
    input=text_input,
    model="text-embedding-ada-002",
    user="website_embeddings.ipynb"
)

In [22]:
#JSON(response)

In [21]:
save_json("website_data/embeddings/response.json",response)

In [25]:
embeddings = [entry["embedding"] for entry in response["data"] ]
#JSON(embeddings)

In [26]:
save_json("website_data/embeddings/embeddings.json",embeddings)

In [2]:
embeddings = load_json("website_data/embeddings/embeddings.json")

# Vector Store

In [3]:
import faiss
import numpy as np

In [5]:
embeddings_np = np.array(embeddings).astype('float32')
embeddings_np

array([[ 0.0071328 ,  0.00672303,  0.00615699, ...,  0.00047228,
        -0.01272376, -0.05311751],
       [ 0.0031525 ,  0.02858355,  0.00448695, ...,  0.00307107,
        -0.02127148, -0.03515112],
       [ 0.00095427,  0.02871195,  0.00421623, ..., -0.00827817,
        -0.01112924, -0.03348833],
       ...,
       [ 0.00623995,  0.02721228, -0.02536341, ..., -0.0013834 ,
        -0.0033462 , -0.04978936],
       [ 0.01363436,  0.03848915, -0.02468762, ...,  0.00132983,
         0.00347379, -0.04359785],
       [ 0.00854609,  0.04833328, -0.01710539, ...,  0.006563  ,
        -0.01306661, -0.04492804]], dtype=float32)

In [12]:
dimension = embeddings_np.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings_np)
print(f"vectors dimension {dimension}")

vectors dimension 1536


In [13]:
faiss.write_index(faiss_index,"website_data/embeddings/faiss_index.index")

In [ ]:
faiss_index = faiss.read_index("website_data/embeddings/faiss_index.index")

# Query

In [14]:
question = "is an Astro island interactive or not ?"

In [15]:
response = openai.Embedding.create(
    input=question,
    model="text-embedding-ada-002",
    user="website_embeddings.ipynb"
)
print(response["usage"])

{
  "prompt_tokens": 8,
  "total_tokens": 8
}


In [22]:
query_embedding = np.array(response["data"][0]["embedding"]).astype('float32')
query_embedding

array([-0.01427942, -0.01084952, -0.01550539, ..., -0.01527863,
       -0.01619988, -0.05011616], dtype=float32)

In [25]:
query_embedding_2d = query_embedding.reshape(1,-1)
query_embedding_2d

array([[-0.01427942, -0.01084952, -0.01550539, ..., -0.01527863,
        -0.01619988, -0.05011616]], dtype=float32)

## search

In [32]:
top_k = 2
distances, indices = faiss_index.search(query_embedding_2d, top_k)

In [38]:
distances[0][0]

0.3331605

In [44]:
print(indices[0])

[0 4]


## Show references

In [33]:
chunks_info = load_json("website_data/export/chunks_infos.json")

In [47]:
messages = []
for i in range(len(indices[0])):
    print(f" * chunk {indices[0][i]} with distance ({distances[0][i]}) from {chunks_info[indices[0][i]]['file_path']} ")
    messages.append({"role":"system", "content":chunks_info[indices[0][i]]['content']})

 * chunk 0 with distance (0.33316048979759216) from concepts/islands 
 * chunk 4 with distance (0.4124778211116791) from concepts/why-astro 


In [48]:
messages.append({"role":"user", "content":question})

In [50]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages
)
print(completion.usage)

{
  "prompt_tokens": 1577,
  "completion_tokens": 69,
  "total_tokens": 1646
}


In [51]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "An Astro island is interactive. It refers to an interactive UI component on an otherwise static page of HTML. The island itself is rendered in isolation, separate from the rest of the static content on the page. You can use any supported UI framework (React, Svelte, Vue, etc.) to render islands in Astro, making them fully interactive."
}
